In [1]:
!pip install tensorflow-gpu==2.0.0.alpha0

     |████████████████████████████████| 332.1MB 51kB/s 
     |████████████████████████████████| 3.0MB 46.4MB/s 
     |████████████████████████████████| 419kB 17.3MB/s 


In [1]:
!pip install pandas-datareader

In [2]:
!pip uninstall -y tensorflow
!pip install tensorflow==2.0

Uninstalling tensorflow-2.0.0:
  Successfully uninstalled tensorflow-2.0.0
  Using cached https://files.pythonhosted.org/packages/46/0f/7bd55361168bb32796b360ad15a25de6966c9c1beb58a8e30c01c8279862/tensorflow-2.0.0-cp36-cp36m-manylinux2010_x86_64.whl


In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random
import pandas_datareader as data_reader
import pandas as pd

from tqdm import tqdm_notebook, tqdm
from collections import deque


/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [0]:
class sAI_Trader():
  def __init__(self, state_size, action_space = 3, model_name = "sAITrader"):
     #Actions are stay, buy and sell
     self.state_size = state_size
     self.action_space = action_space
     self.model_name = model_name
     self.inventory = [] #To store all the stocks bought, to sell only these
     self.memory = deque(maxlen = 2000) #This is for experience replay

     self.gamma = 0.95 #For immediate vs future winnings (reward cashing in)
     self.epsilon = 1.0 #This is exploration vs exploitation - to keep exploring all new states
     self.epsilon_final = 0.01
     self.epsilon_decay = 0.995 
     self.model = self.model_builder()

  def model_builder(self):

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units = 32, activation = 'relu', input_dim = self.state_size)) 
    model.add(tf.keras.layers.Dense(units = 32, activation = 'relu')) 
    model.add(tf.keras.layers.Dense(units = 32, activation = 'relu')) 

    model.add(tf.keras.layers.Dense(units = self.action_space, activation = 'linear')) #Linear activation is used here as the output is changed based on reward which is a real number and not a class

    model.compile(loss = 'mse', optimizer = tf.keras.optimizers.Adam(lr = 0.001))
    return model


  def trade(self, state):

    if random.random() <= self.epsilon:  #First, it is always less than epsilon, and at first we explore
      return random.randrange(self.action_space)

    else:
      actions = self.model.predict(state) #Here, we exploit
      return np.argmax(actions[0]) #Due to the shape of the output   


  def batch_train(self, batch_size):

    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])

    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])

      target = self.model.predict(state)
      target[0][action] = reward

      self.model.fit(state, target, epochs = 1, verbose = 1)

    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay  


In [0]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [0]:
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))

  else:
    return "$ {0:2f}".format(abs(n))  

In [0]:
def dataset_loader(stock_name):

  dataset = data_reader.DataReader(stock_name, data_source = "yahoo")

  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]

  close = dataset['Close']
  return close

In [0]:
def state_creator(data, timestep, window_size):
  
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])

In [0]:
stock_name = "AAPL"
data = dataset_loader(stock_name)

In [0]:
window_size = 10
episodes = 1000

batch_size = 32
data_samples = len(data) - 1

In [0]:
trader = sAI_Trader(window_size)

In [11]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 99        
Total params: 2,563
Trainable params: 2,563
Non-trainable params: 0
_________________________________________________________________


In [0]:
for episode in range(1, episodes + 1):
  
  print("Episode: {}/{}".format(episode, episodes))
  
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory = []
  
  for t in tqdm(range(data_samples)):
    
    action = trader.trade(state)
    
    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0
    
    if action == 1: #Buying
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stocks_price_format(data[t]))
      
    elif action == 2 and len(trader.inventory) > 0: #Selling
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")
    
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
      
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))
    

  0%|          | 0/1258 [00:00<?, ?it/s]

Episode: 1/1000
AI Trader bought:  $ 127.879997
AI Trader bought:  $ 126.599998
AI Trader sold:  $ 127.610001  Profit: - $ 0.269997
AI Trader bought:  $ 127.029999
AI Trader sold:  $ 128.110001  Profit: $ 1.510002
AI Trader bought:  $ 127.500000
AI Trader bought:  $ 124.529999
AI Trader bought:  $ 125.430000
AI Trader sold:  $ 126.599998  Profit: - $ 0.430000
AI Trader sold:  $ 126.440002  Profit: - $ 1.059998
AI Trader sold:  $ 122.570000  Profit: - $ 1.959999
AI Trader bought:  $ 120.070000
AI Trader bought:  $ 123.279999
AI Trader bought:  $ 125.660004
AI Trader sold:  $ 125.610001  Profit: $ 0.180000
AI Trader bought:  $ 126.820000
AI Trader sold:  $ 128.509995  Profit: $ 8.439995
AI Trader sold:  $ 129.619995  Profit: $ 6.339996
AI Trader sold:  $ 132.070007  Profit: $ 6.410004
AI Trader sold:  $ 125.160004  Profit: - $ 1.659996
AI Trader bought:  $ 124.500000
AI Trader bought:  $ 122.769997
AI Trader sold:  $ 122.370003  Profit: - $ 2.129997
1/1 [==============================] -

  3%|▎         | 33/1258 [00:05<03:18,  6.18it/s]

AI Trader sold:  $ 121.300003  Profit: - $ 1.469994
1/1 [==============================] - 0s 2ms/step - loss: 2.7572e-04


  3%|▎         | 34/1258 [00:08<21:03,  1.03s/it]

...

1/1 [==============================] - 0s 1ms/step - loss: 0.0845


  1%|          | 15/1258 [00:46<1:04:00,  3.09s/it]

1/1 [==============================] - 0s 2ms/step - loss: 1.4598


  1%|▏         | 16/1258 [00:49<1:03:39,  3.07s/it]

1/1 [==============================] - 0s 1ms/step - loss: 9.0303e-04


In [0]:
1